In [1]:
from langgraph.prebuilt import create_react_agent
from langchain_mistralai import ChatMistralAI
from langchain_community.tools import tool
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
llm = ChatMistralAI(model_name="mistral-large-latest")

In [3]:
tools = []

In [4]:
data = None
with open('actions.json', 'r') as file:
    data = json.load(file)

In [5]:
def parse_id(id_string):
    return id_string.split('_', 1)[1] if '_' in id_string else id_string

for action in data['actions']:
    exec(action['code'])
    tools.append(globals()[parse_id(action['title'])])

In [6]:
tools

[<function __main__.hello_django()>,
 <function __main__.hello_flask()>,
 <function __main__.hello_python()>,
 <function __main__.hello_react()>,
 <function __main__.hello_universe()>,
 <function __main__.hello_world()>,
 <function __main__.addition(a: int, b: int) -> int>,
 <function __main__.division(a: int, b: int) -> float>,
 <function __main__.multiplication(a: int, b: int) -> int>,
 <function __main__.power(base: int, exponent: int) -> int>,
 <function __main__.subtraction(a: int, b: int) -> int>,
 <function __main__.get_user_list(count: int = 10)>]

In [6]:
@tool
def get_geocoding(city) -> list:
    """
    Retrieve latitude and longitude of a city via API.

    Args:
        city (str): City name.

    Returns:
        list: A list of geocoding data containing 'name', 'latitude', 'longitude', and 'country'.
              Example:
              [
                  {
                      "name": "City Name",
                      "latitude": 12.34,
                      "longitude": 56.78,
                      "country": "Country Name"
                  }
              ]
              
              Prints error message and status code if the request fails.
    """

    import requests
    api_url = 'https://api.api-ninjas.com/v1/geocoding?city={}'.format(city)
    response = requests.get(api_url, headers={'X-Api-Key': "meaUO1Ucu4ZTlk7qtQMibA==cpyHoWVnOLLm6QW2"})
    if response.status_code == requests.codes.ok:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)

In [7]:
tools = [get_geocoding]
get_geocoding.__doc__

'Tool that can operate on any number of inputs.'

In [8]:
agent = create_react_agent(llm, tools)

In [12]:
from langchain_core.messages import HumanMessage
import json
import asyncio

def process_stream(stream):
    try:
        for chunk in stream:
            print("Received chunk:", chunk)
            if hasattr(chunk, 'content'):
                print(chunk.content, end='', flush=True)
            elif isinstance(chunk, dict):
                print(json.dumps(chunk, indent=2))
            else:
                print(f"Unexpected chunk type: {type(chunk)}")
    except Exception as e:
        print(f"An error occurred: {e}")

async def main():
    stream = agent.stream({"messages": [HumanMessage(content="From the list of users, give me the email and name of first user")]})
    # Use run_in_executor to run the synchronous function in a separate thread
    await asyncio.get_event_loop().run_in_executor(None, process_stream, stream)

await main()

In [17]:
await main()

TypeError: object async_generator can't be used in 'await' expression

In [9]:
agent.invoke({"messages": [*result["messages"], HumanMessage(content="What is the name of the 5 the user?")]})

{'messages': [HumanMessage(content='From the list of users, give me the email and name of first user', id='4ad9e0f6-c4f4-455e-a543-943f438df510'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'p1znGB4z7', 'function': {'name': 'get_user_list', 'arguments': '{"count": 1}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 797, 'total_tokens': 819, 'completion_tokens': 22}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-8f6be88a-e2c1-4b0f-8fa0-7b0ce0f59868-0', tool_calls=[{'name': 'get_user_list', 'args': {'count': 1}, 'id': 'p1znGB4z7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 797, 'output_tokens': 22, 'total_tokens': 819}),
  ToolMessage(content='[{"id": 0, "name": "abhishek 0", "email": "abhishek0@mail.com"}]', name='get_user_list', id='6f3b20b2-6a67-4c53-8823-0552f308d284', tool_call_id='p1znGB4z7'),
  AIMessage(content='Here is the details of the first user:\n- Name: abhishek 0\n- Email: abhishek0@mail.com', response_m

In [10]:
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

In [9]:
previous_messages = []
for message in result["messages"]:
    if isinstance(message, AIMessage):
        if not message.content:
            continue
        previous_messages.append(AIMessage(content=message.content))
    elif isinstance(message, HumanMessage):
        previous_messages.append(message)
    elif isinstance(message, ToolMessage):
        previous_messages.append(AIMessage(content=json.dumps(message.content)))

previous_messages

NameError: name 'AIMessage' is not defined

In [12]:

agent.invoke({"messages": [*previous_messages, HumanMessage(content="What is the email user with is 0 again")]})

{'messages': [HumanMessage(content='From the list of users, give me the email and name of first user', id='4ad9e0f6-c4f4-455e-a543-943f438df510'),
  AIMessage(content='"[{\\"id\\": 0, \\"name\\": \\"abhishek 0\\", \\"email\\": \\"abhishek0@mail.com\\"}]"', id='ed39fe4c-616f-40b4-ac6b-11d75ea76189'),
  AIMessage(content='Here is the details of the first user:\n- Name: abhishek 0\n- Email: abhishek0@mail.com', id='4397df32-2fec-49a4-ab20-e345c215494e'),
  HumanMessage(content='What is the email user with is 0 again', id='bb9876af-6919-468b-96ad-fad8cabe7906'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'Cxx8MSvtA', 'function': {'name': 'get_user_list', 'arguments': '{"count": 10}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 879, 'total_tokens': 902, 'completion_tokens': 23}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-4431dcd1-5742-47c4-a94d-a50ded0aba87-0', tool_calls=[{'name': 'get_user_list', 'args': {'count': 10}, 'id': 

In [23]:
result = None
result = agent.invoke({"messages": [HumanMessage(content="Get me first 20 Users list")]})

for message in result["messages"]:
    print(message.content)

Get me first 20 Users list

[{"id": 0, "name": "abhishek 0", "email": "abhishek0@mail.com"}, {"id": 1, "name": "abhishek 1", "email": "abhishek1@mail.com"}, {"id": 2, "name": "abhishek 2", "email": "abhishek2@mail.com"}, {"id": 3, "name": "abhishek 3", "email": "abhishek3@mail.com"}, {"id": 4, "name": "abhishek 4", "email": "abhishek4@mail.com"}, {"id": 5, "name": "abhishek 5", "email": "abhishek5@mail.com"}, {"id": 6, "name": "abhishek 6", "email": "abhishek6@mail.com"}, {"id": 7, "name": "abhishek 7", "email": "abhishek7@mail.com"}, {"id": 8, "name": "abhishek 8", "email": "abhishek8@mail.com"}, {"id": 9, "name": "abhishek 9", "email": "abhishek9@mail.com"}, {"id": 10, "name": "abhishek 10", "email": "abhishek10@mail.com"}, {"id": 11, "name": "abhishek 11", "email": "abhishek11@mail.com"}, {"id": 12, "name": "abhishek 12", "email": "abhishek12@mail.com"}, {"id": 13, "name": "abhishek 13", "email": "abhishek13@mail.com"}, {"id": 14, "name": "abhishek 14", "email": "abhishek14@mail.com

In [10]:
get_geocoding("london", "uk")[0]

{'name': 'London',
 'latitude': 51.5073219,
 'longitude': -0.1276474,
 'country': 'GB',
 'state': 'England'}

### Checking For Session Queries

In [5]:
from langchain_core.messages import HumanMessage, AIMessage

result = agent.invoke({"messages": [HumanMessage(content="Who is Narendra Modi?")]})
result

{'messages': [HumanMessage(content='Who is Narendra Modi?', id='4c146c78-5b7c-452e-b5b5-dff75bbf4185'),
  AIMessage(content="Narendra Modi is an Indian politician who has been serving as the Prime Minister of India since 2014. Here are some key points about him:\n\n1. **Early Life and Education**: Narendra Damodardas Modi was born on September 17, 1950, in Vadnagar, Gujarat. He holds a Bachelor's degree in political science from the School of Open Learning, University of Delhi, and a Master's degree in political science from Gujarat University.\n\n2. **Political Career**: Modi began his political career as a member of the Rashtriya Swayamsevak Sangh (RSS) and joined the Bharatiya Janata Party (BJP) in 1985. He rose through the ranks, holding various positions within the party.\n\n3. **Chief Minister of Gujarat**: Modi served as the Chief Minister of Gujarat from 2001 to 2014, making him the longest-serving Chief Minister of the state. His tenure is known for significant economic growth

In [6]:
result['messages'].append(HumanMessage(content="What is his age, again?"))

result

{'messages': [HumanMessage(content='Who is Narendra Modi?', id='4c146c78-5b7c-452e-b5b5-dff75bbf4185'),
  AIMessage(content="Narendra Modi is an Indian politician who has been serving as the Prime Minister of India since 2014. Here are some key points about him:\n\n1. **Early Life and Education**: Narendra Damodardas Modi was born on September 17, 1950, in Vadnagar, Gujarat. He holds a Bachelor's degree in political science from the School of Open Learning, University of Delhi, and a Master's degree in political science from Gujarat University.\n\n2. **Political Career**: Modi began his political career as a member of the Rashtriya Swayamsevak Sangh (RSS) and joined the Bharatiya Janata Party (BJP) in 1985. He rose through the ranks, holding various positions within the party.\n\n3. **Chief Minister of Gujarat**: Modi served as the Chief Minister of Gujarat from 2001 to 2014, making him the longest-serving Chief Minister of the state. His tenure is known for significant economic growth

In [7]:
res = agent.invoke(result)
res

{'messages': [HumanMessage(content='Who is Narendra Modi?', id='4c146c78-5b7c-452e-b5b5-dff75bbf4185'),
  AIMessage(content="Narendra Modi is an Indian politician who has been serving as the Prime Minister of India since 2014. Here are some key points about him:\n\n1. **Early Life and Education**: Narendra Damodardas Modi was born on September 17, 1950, in Vadnagar, Gujarat. He holds a Bachelor's degree in political science from the School of Open Learning, University of Delhi, and a Master's degree in political science from Gujarat University.\n\n2. **Political Career**: Modi began his political career as a member of the Rashtriya Swayamsevak Sangh (RSS) and joined the Bharatiya Janata Party (BJP) in 1985. He rose through the ranks, holding various positions within the party.\n\n3. **Chief Minister of Gujarat**: Modi served as the Chief Minister of Gujarat from 2001 to 2014, making him the longest-serving Chief Minister of the state. His tenure is known for significant economic growth